# ADAPTIC preprocessor

This scripts generates a D-Frame system input file _.dat_ for analysis in __ADAPTIC__.

Refer to _analysis.ipynb_ , located in the same folder, to learn about the details of the model and the analysis results.

## Input

Use the code block below to define basic analysis options.

In [1]:
project_name = '2plt3col'
save_file = True # generates *.dat file
analysis_2d = True # 3d analysis if False
internal_bay = True # Edge bay if False
save_pickle = True

In [2]:
# System geometry
case = 2
# 0 - continuous beam without joints
# 2 - double plate

# Column removed
col_rem = 3

## Initialisation

Import necessary libraries, define functions and intialise `all_lines` variable to store each line of text to be printed in _*.dat_ file.

In [3]:
import numpy as np
import pandas as pd
from datetime import datetime as day
from math import log10, floor
import sys
sys.path.insert(1, '../../modules')
import common
import materials as mat
import sections as sect
import restraints as restr
import joints

rigid = 'rigid'

# Functions
def add(line): return np.vstack((all_lines, line))

def hash_tag(lst): return ['#']*len(lst)

def obj_loc(obj_lst, val, att):
    for i in range(len(obj_lst)):
        if getattr(obj_lst[i], att) == val:
            return obj_lst[i]
    print('No object has an attribute {0} with a value {1}'.format(att, val))

# Init
all_lines = np.array("# Init")
global_print = True # prevent printing and plotting from other modules

# Write file info
line = common.str_joint(["#",project_name,str(day.today())])
all_lines = add(line)
if analysis_2d:
    all_lines = add("analysis 2d statics")
else:
    all_lines = add("analysis 3d statics")

## Define geometry

In [4]:
bay_no = 4 # number of bays
bay_len = 9000 # column-to-column spacing
plank_s = 3000 # width of a single plank
total_length = bay_len * bay_no 
mesh_size = 250 # beam mesh size

joint_loc = [bay_len + 1500,2*bay_len + 1500,3*bay_len + 1500] # joint locations

## Define materials

Refer to _materials.ipynb_ to learn how the material properties are defined and calulcated.

Concrete C50/60 is used for the precast sections and C90/100 grout to fill the voids and recesses. Characteristic cylinder compressive strengths from _Eurocode 2_ are used in the model.

Steel plate and reinforcement properties are based on tensile tests carried out at Imperial College London in March 2019.

In [5]:
all_lines = add('#')
all_lines = add('materials')
all_lines = add('mat.name model properties')
# Instatiate all con1 materials
grout = mat.con1('grout', 90, 20)
concr = mat.con1('C50_60', 50, 250)
# Instatiate all stl1 materials
reinf = mat.stl1('reinf', 205000, 550, 650, 0.1)
plt = mat.stl1('plt15mm', 171000, 390, 550, 0.1)
# Group al meterials in material_lst
material_lst = [concr,grout,reinf,plt]

all_lines = add(concr.adaptic_print())
all_lines = add(grout.adaptic_print())
all_lines = add(reinf.adaptic_print())
all_lines = add(plt.adaptic_print())

## Define sections

Refer to _sections.ipynb_ to learn how the material properties are defined and calulcated to be incorporated in __ADAPTIC__.

### rcts sections

Reinforced concrete T-section.

There are five beam sections in the model (*B1* - *B5*), which are mapped to either internal or edge sections as follows:

|Model section ID|Internal section ID|Edge section ID|
|:-:|:-:|:-:|
|sectB1|Section 1|Section 6|
|sectB2|Section 2|Section 7|
|sectB3|Section 3|Section 8|
|sectB4|Section 4|Section 9|
|sectB5|Section 5|Section 10|

In [6]:
%run ../../modules/sections.ipynb
section_rcts_lst = []
all_lines = add('#')
all_lines = add('sections')
all_lines = add('type = rcts')
all_lines = add('sec.name mat.name dimensions')

if internal_bay:
    # Beam Section 1
    section = beam_section1
    common.setattrs(section, ID = 'sectB1', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 2
    section = beam_section2
    common.setattrs(section, ID = 'sectB2', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 3
    section = beam_section3
    common.setattrs(section, ID = 'sectB3', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 4
    section = beam_section4
    common.setattrs(section, ID = 'sectB4', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 5
    section = beam_section5
    common.setattrs(section, ID = 'sectB5', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
else:
    # Beam Section 6
    section = beam_section6
    common.setattrs(section, ID = 'sectB1', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 7
    section = beam_section7
    common.setattrs(section, ID = 'sectB2', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 8
    section = beam_section8
    common.setattrs(section, ID = 'sectB3', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 9
    section = beam_section9
    common.setattrs(section, ID = 'sectB4', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)
    # Beam Section 10
    section = beam_section10
    common.setattrs(section, ID = 'sectB5', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rcts_lst.append(section)

for i in section_rcts_lst:
    all_lines = add(i.adaptic_print())

### rccs sections (Sections 11-14)

Reinforced concrete column section.

There are two column sections in the model (*C1* - *C2*), which are mapped to either internal or edge sections as follows:

|Model section ID|Internal bay column ID|Edge bay column ID|
|:-:|:-:|:-:|
|sectC1|Column Section 1|Column Section 3|
|sectC2|Column Section 2|Column Section 4|

In [7]:
section_rccs_lst = []
all_lines = add('type = rccs')
all_lines = add('sec.name mat.name dimensions')

if internal_bay:
    # Column Section 1 - internal column
    section = col_section1
    common.setattrs(section, ID = 'sectC1', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rccs_lst.append(section)
    # Column Section 2 - edge type 2 column
    section = col_section2
    common.setattrs(section, ID = 'sectC2', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rccs_lst.append(section)
else:
    # Column Section 3 - edge type 2 column
    section = col_section3
    common.setattrs(section, ID = 'sectC1', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rccs_lst.append(section)
    # Column Section 4 - corner column
    section = col_section4
    common.setattrs(section, ID = 'sectC2', reinf_mat = reinf.ID, unconf_mat = concr.ID, conf_mat = concr.ID)
    section_rccs_lst.append(section)

for i in section_rccs_lst:
    all_lines = add(i.adaptic_print())

### rss sections

In [8]:
section_rss_lst = []
all_lines = add('type = rss')
all_lines = add('sec.name mat.name dimensions')

section = sect.rss('tPlate', plt.ID, 250, 15)
section_rss_lst.append(section)
section = sect.rss('bPlate', plt.ID, 250, 15)
section_rss_lst.append(section)

for i in section_rss_lst:
    all_lines = add(i.adaptic_print())

In [9]:
#group all sections as section_lst
section_lst = [section_rccs_lst, section_rss_lst]

## Groups

### cbp2

In [10]:
sections_lst = [section_rcts_lst, section_rccs_lst, section_rss_lst]
sec_names = np.array([i.ID for sub_l in sections_lst for i in sub_l])

monitor_pts = 200
all_lines = add('#')
all_lines = add('groups')
all_lines = add('type.of.element = cbp2')
all_lines = add('grp.name sec.name monitoring.points')


cbp2_lst = [['g'+i,i,monitor_pts] for i in sec_names]
for i in cbp2_lst:
    all_lines = add(common.str_joint(i))

### lnk2

In [11]:
all_lines = add('type.of.element = lnk2')
all_lines = add('grp.name stiffness.parameters')
lnk_lst = []

x = rigid
y = rigid
rz = rigid
lnk_lst.append(common.param2adap('glnkRig', [x, y, rz]))

for i in lnk_lst:
    all_lines = add(i.printout)

### jel2

In [12]:
%run ../../modules/restraints.ipynb 

if internal_bay:
    sup_int = col1
    sup_end = col2
else:
    sup_int = col3
    sup_end = col4
all_lines = add('type.of.element = jel2')
all_lines = add('grp.name curve.types parameters')
jel_lst = []

# rigid jel
jel_lst.append(common.param2adap('gjelRig', [rigid, rigid, rigid]))

# support jel elements
## pick the type of horizontal restraints:
col1.kh = col1.kh_lin
col3.kh = col3.kh_lin

jel_lst.append(common.param2adap('gjelSup1', sup_int.combined()))
jel_lst.append(common.param2adap('gjelSup2', sup_int.combined()))
jel_lst.append(common.param2adap('gjelSup3', sup_int.combined()))
jel_lst.append(common.param2adap('gjelSup4', sup_int.combined()))
jel_lst.append(common.param2adap('gjelSup5', sup_int.combined()))

if col_rem != 0:
    x = ['lin', 0.1]
    y = ['lin', 0.1]
    rz = ['lin', 0.1]
    jel_lst[col_rem] = common.param2adap('gjelSup'+str(col_rem), [x, y, rz])

# joint jel elements
if case != 0:
    x = ['lin', 0.1]
    y = ['contact', 0.01, 99999]
    rz = ['lin', 0.1]
    jel_lst.append(common.param2adap('gjelHL', [x, y, rz]))
    y = ['lin', 0.1]
    x = ['astr', 0.1,0,0,1,0,2.00E+09,0,0,-0.1,2.00E+08]
    jel_lst.append(common.param2adap('gjelTCon', [x, y, rz]))
    y = ['lin', 0.1]
    x = ['astr', 0.1,0,0,1,0,2.00E+09,0,0,-20,2.00E+08]
    jel_lst.append(common.param2adap('gjelBCon', [x, y, rz]))
    y = ['contact', 0.01,99999]
    x = ['lin', 0.1]
    jel_lst.append(common.param2adap('gjelTplt', [x, y, rz]))
    y = ['astr', 2.00E+09,0,0,0.01,2.00E+08,0.1,0,0,-1,0]
    x = ['lin', 0.1]
    jel_lst.append(common.param2adap('gjelBplt', [x, y, rz]))
    
for i in jel_lst:
    all_lines = all_lines = add(i.printout)

In [13]:
#group all section groups as group_lst
group_lst = [cbp2_lst,lnk_lst,jel_lst]

## Structural nodes

### Beam nodes

In [14]:
all_lines = add('#')
all_lines = add('structural.nodal.coordinates')
all_lines = add('nod.n x y')

no_beam_nodes = int(total_length / mesh_size) +1
nodes_beamX = [mesh_size * i for i in range(no_beam_nodes)]
nodes_beamY = [0] * no_beam_nodes
nodes_beamName = common.name_gen("",nodes_beamX)
nodes_beamID = common.name_gen("beam node ",nodes_beamX)
nodes_beam = pd.DataFrame([nodes_beamName,nodes_beamX, nodes_beamY, hash_tag(nodes_beamX),nodes_beamID]).transpose()
nodes_beam.columns = ["name",'x','y','#','ID']

nodes_rib = np.arange(0,total_length+plank_s,plank_s).astype(int)

indx = 0
for i in nodes_rib:
    nodes_beam.at[common.df_index(nodes_beam,i,'x'), 'ID'] = 'beam node rib '+ str(indx)
    indx += 1

if case != 0:
    for i in range(len(joint_loc)):
        idx = nodes_beam.index[nodes_beam['x'] == joint_loc[i]].tolist()[0]
        nodes_beam.at[idx-2,'ID']='beam node joint '+str(i+1)+' node a'
        nodes_beam.at[idx+2,'ID']='beam node joint '+str(i+1)+' node b'
        nodes_beam = nodes_beam.drop(labels=[idx-1,idx,idx+1],axis=0)
nodes_beam = nodes_beam.reset_index(drop=True)

### Column & support nodes

In [15]:
nodes_support_x = np.arange(0,total_length + bay_len,bay_len).astype(int)

# update ID of beam nodes neighbouring supports
beam_supp_ID = []
for i,val in enumerate(nodes_support_x):
    idx = common.df_index(nodes_beam,val,'x')
    beam_supp_ID.append(idx)
    nodes_beam.xs(int(idx))['ID']='beam node support ' + str(i+1)

nodes_supportID = common.name_gen("support node ",nodes_support_x)
nodes_supportName = common.name_gen("",nodes_support_x, start_ID = 301)
nodes_support = pd.DataFrame([nodes_supportName, nodes_support_x,np.zeros(bay_no + 1).astype(int),
                              hash_tag(nodes_supportID),nodes_supportID]).transpose()
nodes_support.columns = ['name', 'x','y','#','ID']
nodes_colID = common.name_gen("column node ",nodes_support_x)
nodes_colName = common.name_gen("",nodes_support_x, start_ID = 201)
nodes_col = pd.DataFrame([nodes_colName, nodes_support_x,np.repeat(-1000, bay_no + 1),
                          hash_tag(nodes_colID),nodes_colID]).transpose()
nodes_col.columns = ['name','x','y','#','ID']
nodes_col_ref = nodes_col # used as a referenece of all column nodes before removing the column
if col_rem != 0:
    nodes_col = nodes_col.drop([col_rem - 1])

### Joint nodes

In [16]:
# instatiate joint objects and hold in joint_model
joint_model = []
if case != 0:
    startNode = 401
    startCBP = 301
    startLNK = 801
    for i in range(len(joint_loc)):
        nodeA = common.df_value(nodes_beam,'beam node joint '+str(i+1)+' node a','ID', 'name')
        nodeB = common.df_value(nodes_beam,'beam node joint '+str(i+1)+' node b','ID', 'name')
        joint_model.append(joints.HL_TBplt('joint '+str(i+1), joint_loc[i], startNode, section_rcts_lst, 
                                           section_rss_lst, lnk_lst, startLNK, nodeA, nodeB, startCBP))
        startNode = int(joint_model[i].nodes.tail(1)['name'].tolist()[0])+1
        startCBP = int(joint_model[i].cbp.tail(1)['name'].tolist()[0])+1
        startLNK = int(joint_model[i].lnk.tail(1)['name'].tolist()[0])+1
    nodes_joint = pd.concat([joint_model[0].nodes,joint_model[1].nodes,joint_model[2].nodes])
else:
    nodes_joint=pd.DataFrame([])
nodes_joint

,name,x,y,#,ID
0,401,10500.0,-9.0,#,joint 1 node 1
1,402,10750.0,-9.0,#,joint 1 node 2
2,403,10565.0,22.0,#,joint 1 node 3
3,404,10565.0,22.0,#,joint 1 node 4
4,405,10685.0,22.0,#,joint 1 node 5
...,...,...,...,...,...
71,624,28000.0,-9.0,#,joint 3 node 88
72,625,28250.0,-9.0,#,joint 3 node 89
73,626,28500.0,-9.0,#,joint 3 node 90
74,627,28750.0,-9.0,#,joint 3 node 91


### Assemble the nodes

In [17]:
nodes = pd.concat([nodes_beam, nodes_col, nodes_support, nodes_joint])
nodes = nodes.reset_index(drop=True)
for i in range(len(nodes)):
    all_lines = add(nodes.loc[[i]].to_string(header=False, index=False))
nodes

,name,x,y,#,ID
0,1,0,0,#,beam node support 1
1,2,250,0,#,beam node 2
2,3,500,0,#,beam node 3
3,4,750,0,#,beam node 4
4,5,1000,0,#,beam node 5
...,...,...,...,...,...
368,624,28000,-9,#,joint 3 node 88
369,625,28250,-9,#,joint 3 node 89
370,626,28500,-9,#,joint 3 node 90
371,627,28750,-9,#,joint 3 node 91


## Non-structural nodes

In [18]:
all_lines = add('#')
all_lines = add('non.structural.nodal.coordinates')
all_lines = add('nod.n x y')

if case == 0:
    non_st_nodes = np.array([[999999, 0, '#','sectB2 centroid']])
else:
    non_st_nodes = pd.DataFrame([
        [999999, 0, '#','sectB2 centroid'],
        [999999, joint_model[0].plt_top_cetr, '#','top plate centroid'],
        [999999, joint_model[0].lap_Y, '#','lap centroid'],
        [999999, joint_model[0].sect2_top, '#','sectB3 top'],
        [999999, joint_model[0].sect3_centr, '#','sectB3 centroid'],
        [999999, joint_model[0].compr_centr, '#','bottom contact'],
        [999999, joint_model[0].top_grout_centr, '#','grout centroid'],
        [999999, joint_model[0].plt_bot_cetr, '#','bottom plate centroid'],
        [999999, joint_model[0].compr_centr, '#','bottom centroid in compression']
    ])

non_st_nodesName = common.name_gen("nst",non_st_nodes)
non_st_nodes.columns = ['x','y', '#','ID']
non_st_nodes.insert(0, "name", non_st_nodesName, True)

for i in range(len(non_st_nodes)):
    all_lines = add(non_st_nodes.loc[[i]].to_string(header=False, index=False))
non_st_nodes

,name,x,y,#,ID
0,nst1,999999,0.0,#,sectB2 centroid
1,nst2,999999,420.5,#,top plate centroid
2,nst3,999999,22.0,#,lap centroid
3,nst4,999999,428.0,#,sectB3 top
4,nst5,999999,-9.0,#,sectB3 centroid
5,nst6,999999,-224.0,#,bottom contact
6,nst7,999999,338.0,#,grout centroid
7,nst8,999999,-329.5,#,bottom plate centroid
8,nst9,999999,-224.0,#,bottom centroid in compression


## Restraints

In [19]:
all_lines = add('#')
all_lines = add('restraints')
all_lines = add('nod.n direction')
ID = common.name_gen("support ",nodes_supportName)
restraints = np.vstack((nodes_supportName,['x+y+rz']*len(nodes_supportName),hash_tag(nodes_supportName),ID)).T
restraints = pd.DataFrame(restraints)
restraints.columns = ['name','direction','#','ID']
for i in range(len(restraints)):
    all_lines = add(restraints.loc[[i]].to_string(header=False, index=False))
restraints

,name,direction,#,ID
0,301,x+y+rz,#,support 1
1,302,x+y+rz,#,support 2
2,303,x+y+rz,#,support 3
3,304,x+y+rz,#,support 4
4,305,x+y+rz,#,support 5


## Element connectivity

### cbp elements - beams

In [20]:
all_lines = add('#')
all_lines = add('element.connectivity')
all_lines = add('elm.name grp.name nod.')
all_lines = add('# name node1, node2, start, end, ID')

#for i in range(0,len(nodes_beam)-1):
#    a = nodes_beam.loc[i+1]['name']

cbp_beam = [[nodes_beam.loc[i+1]['name'],nodes_beam.loc[i]['name'],'#', nodes_beam.loc[i]['x'], 
             nodes_beam.loc[i+1]['x'], "beam elm "+str(i+1) ] for i in range(0,len(nodes_beam)-1)]
cbp_beam = pd.DataFrame(cbp_beam)
cbp_beam.columns = ['node 1','node 2','#','start','end','ID']

sect_type = []
for i in range(len(cbp_beam)):
    position = cbp_beam.loc[i]['start']+(cbp_beam.loc[i]['end']-cbp_beam.loc[i]['start'])/2
    for j in sect_dist:
        if j[1] <= position <= j[2]:
            sect_type.append(j[0])
            break
cbp_beam.insert(0, "section", sect_type, True)
cbp_beam.insert(0, "name", common.name_gen("",cbp_beam), True)
if case != 0: # remove cbp elements at joint location
    for i in range(len(joint_loc)):
        idx = cbp_beam.index[cbp_beam['start'] == joint_loc[i]-500].tolist()[0]
        cbp_beam = cbp_beam.drop(labels=idx,axis=0)
cbp_beam.head(50)

,name,section,node 1,node 2,#,start,end,ID
0,1,gsectB1,2,1,#,0,250,beam elm 1
1,2,gsectB1,3,2,#,250,500,beam elm 2
2,3,gsectB1,4,3,#,500,750,beam elm 3
3,4,gsectB1,5,4,#,750,1000,beam elm 4
4,5,gsectB1,6,5,#,1000,1250,beam elm 5
5,6,gsectB1,7,6,#,1250,1500,beam elm 6
6,7,gsectB1,8,7,#,1500,1750,beam elm 7
7,8,gsectB1,9,8,#,1750,2000,beam elm 8
8,9,gsectB1,10,9,#,2000,2250,beam elm 9
9,10,gsectB1,11,10,#,2250,2500,beam elm 10


### cbp elements - columns

In [21]:
cbp_col = pd.DataFrame([['g'+section_rccs_lst[1].ID, nodes_col_ref.loc[0]['name'], 
            nodes_support.loc[0]['name'],'#',"col elm 1"],
          ['g'+section_rccs_lst[0].ID, nodes_col_ref.loc[1]['name'], 
           nodes_support.loc[1]['name'],'#',"col elm 2"],
          ['g'+section_rccs_lst[0].ID, nodes_col_ref.loc[2]['name'], 
           nodes_support.loc[2]['name'],'#',"col elm 3"],
          ['g'+section_rccs_lst[0].ID, nodes_col_ref.loc[3]['name'], 
           nodes_support.loc[3]['name'],'#',"col elm 4"],
          ['g'+section_rccs_lst[1].ID, nodes_col_ref.loc[4]['name'], 
           nodes_support.loc[4]['name'],'#',"col elm 5"]])
if col_rem != 0:
    cbp_col = cbp_col.drop([col_rem - 1])
cbp_col.columns= ['section','node 1','node 2','#','ID']
cbp_col.insert(0, "name", common.name_gen("",cbp_col,200), True)
cbp_col

,name,section,node 1,node 2,#,ID
0,200,gsectC2,201,301,#,col elm 1
1,201,gsectC1,202,302,#,col elm 2
3,202,gsectC1,204,304,#,col elm 4
4,203,gsectC2,205,305,#,col elm 5


### cbp elements - joints

In [22]:
cbp_joint = pd.concat([joint_model[0].cbp,joint_model[1].cbp,joint_model[2].cbp])
cbp_joint

,name,section,node 1,node 2,#,ID
0,301,gsectB3,473,472,#,joint 1 cbp sect3 1
1,302,gsectB3,401,473,#,joint 1 cbp sect3 2
2,303,gsectB3,476,402,#,joint 1 cbp sect3 3
3,304,gsectB4,417,416,#,joint 1 cbp sect4 1
4,305,gsectB4,418,417,#,joint 1 cbp sect4 2
...,...,...,...,...,...,...
36,419,gbPlate,616,617,#,joint 3 cbp bot plt 12
37,420,gbPlate,617,618,#,joint 3 cbp bot plt 13
38,421,gbPlate,618,619,#,joint 3 cbp bot plt 14
39,422,gbPlate,619,620,#,joint 3 cbp bot plt 15


### Assemble cbp elements

In [23]:
all_lines = add('# cbp elements')
elm_cbp = pd.concat([cbp_beam, cbp_col,cbp_joint])
elm_cbp = elm_cbp.reset_index(drop=True)
for i in range(len(elm_cbp)):
    all_lines = add(elm_cbp.loc[[i]].to_string(header=False, index=False))
elm_cbp

,name,section,node 1,node 2,#,start,end,ID
0,1,gsectB1,2,1,#,0.0,250.0,beam elm 1
1,2,gsectB1,3,2,#,250.0,500.0,beam elm 2
2,3,gsectB1,4,3,#,500.0,750.0,beam elm 3
3,4,gsectB1,5,4,#,750.0,1000.0,beam elm 4
4,5,gsectB1,6,5,#,1000.0,1250.0,beam elm 5
...,...,...,...,...,...,...,...,...
254,419,gbPlate,616,617,#,NaN,NaN,joint 3 cbp bot plt 12
255,420,gbPlate,617,618,#,NaN,NaN,joint 3 cbp bot plt 13
256,421,gbPlate,618,619,#,NaN,NaN,joint 3 cbp bot plt 14
257,422,gbPlate,619,620,#,NaN,NaN,joint 3 cbp bot plt 15


### jel elements

#### jel support elements

In [24]:
nst_sup = common.df_value(non_st_nodes,'sectB2 centroid','ID', 'name')
jel_sup = pd.DataFrame([['gjelSup1',common.df_value(nodes_beam,'beam node support 1','ID', 'name'), 
            nodes_support.loc[common.df_index(nodes_support,'support node 1','ID')]['name'],nst_sup,'#','jel support 1'],
          ['gjelSup2',common.df_value(nodes_beam,'beam node support 2','ID', 'name'), 
            nodes_support.loc[common.df_index(nodes_support,'support node 2','ID')]['name'],nst_sup,'#','jel support 2'],
          ['gjelSup3',common.df_value(nodes_beam,'beam node support 3','ID', 'name'), 
            nodes_support.loc[common.df_index(nodes_support,'support node 3','ID')]['name'],nst_sup,'#','jel support 3'],
          ['gjelSup4',common.df_value(nodes_beam,'beam node support 4','ID', 'name'), 
            nodes_support.loc[common.df_index(nodes_support,'support node 4','ID')]['name'],nst_sup,'#','jel support 4'],
          ['gjelSup5',common.df_value(nodes_beam,'beam node support 5','ID', 'name'), 
            nodes_support.loc[common.df_index(nodes_support,'support node 5','ID')]['name'],nst_sup,'#','jel support 5']])
jel_sup.columns = ['section','node 1','node 2','nst node','#','ID']
Name = common.name_gen("",jel_sup, start_ID = 501)
jel_sup.insert(0, "name", Name, True)

#### jel joint elements

In [25]:
if case != 0:
    startJEL = int(jel_sup.tail(1)['name'].tolist()[0])+1
    jel_joint=pd.DataFrame([])
    for i in joint_model:
        i.jel_elements(jel_lst,non_st_nodes,startJEL)
        jel_joint = pd.concat([jel_joint,i.jel])
        startJEL = int(i.jel.tail(1)['name'].tolist()[0])+1

#### assemble jel elements

In [26]:
all_lines = add('# jel elements')
elm_jel = pd.concat([jel_sup,jel_joint])
elm_jel = elm_jel.reset_index(drop=True)
for i in range(len(elm_jel)):
    all_lines = add(elm_jel.loc[[i]].to_string(header=False, index=False))
elm_jel

,name,section,node 1,node 2,nst node,#,ID
0,501,gjelSup1,1,301,nst1,#,jel support 1
1,502,gjelSup2,37,302,nst1,#,jel support 2
2,503,gjelSup3,73,303,nst1,#,jel support 3
3,504,gjelSup4,109,304,nst1,#,jel support 4
4,505,gjelSup5,145,305,nst1,#,jel support 5
5,506,gjelHL,403,404,nst3,#,"joint 1 jel J1, bearing pad 1"
6,507,gjelHL,405,406,nst3,#,"joint 1 jel J2, bearing pad 2"
7,508,gjelBCon,470,413,nst9,#,"joint 1 jel J3, bottom centroid in compression"
8,509,gjelTCon,420,447,nst7,#,"joint 1 jel J4, grout centroid"
9,510,gjelTplt,423,446,nst4,#,joint 1 jel J5


### lnk elements

In [27]:
if case != 0:
    all_lines = add('# lnk elements')
    joint_lnk = pd.concat([joint_model[0].lnk,joint_model[1].lnk,joint_model[2].lnk]).reset_index(drop=True)
    
    elm_lnk = joint_lnk
    for i in range(len(joint_lnk)):
        all_lines = add(joint_lnk.loc[[i]].to_string(header=False, index=False))
else:
    joint_lnk = pd.DataFrame([])
joint_lnk

,name,section,node 1,node 2,#,ID
0,801,glnkRig,401,407,#,joint 1 lnk RL1
1,802,glnkRig,410,470,#,joint 1 lnk RL2
2,803,glnkRig,408,403,#,joint 1 lnk RL3
3,804,glnkRig,401,415,#,joint 1 lnk RL4
4,805,glnkRig,404,417,#,joint 1 lnk RL5
...,...,...,...,...,...,...
88,889,glnkRig,600,613,#,joint 3 lnk RL27
89,890,glnkRig,622,564,#,joint 3 lnk RL28
90,891,glnkRig,623,566,#,joint 3 lnk RL29
91,892,glnkRig,624,113,#,joint 3 lnk RL30


## Loading

In [28]:
%run ../../modules/loading.ipynb

all_lines = add('#')
all_lines = add('applied.loading')
all_lines = add('proportional.loads')
all_lines = add('nod.name direction type value')

SW = w1 * mesh_size

load = pd.DataFrame([[nodes_beam.loc[i]['name'],'y','force',SW,'#',
                      nodes.loc[i]['ID']] for i in range(len(nodes_beam))])
load.columns = ['node','direction','type','value','#','ID']

for i in range(1,len(nodes_rib)-1):
    try:
        load.at[common.df_index(load,'beam node rib '+str(i),'ID'), 'value'] = F1 + SW
    except:
        pass

load.at[common.df_index(load,'beam node support 1','ID'), 'value'] = F2 + SW/2
load.at[common.df_index(load,'beam node support 2','ID'), 'value'] = F3 + SW
load.at[common.df_index(load,'beam node support 3','ID'), 'value'] = F3 + SW
load.at[common.df_index(load,'beam node support 4','ID'), 'value'] = F3 + SW
load.at[common.df_index(load,'beam node support 5','ID'), 'value'] = F2 + SW/2

if case != 0:
    load_joint = pd.DataFrame([])
    for i in joint_model:
        load_temp = pd.DataFrame([
            [common.df_value(i.nodes,i.ID+' node 89','ID','name'),'y','force',SW,'#',str(i.ID)+' node 89 load'],
            [common.df_value(i.nodes,i.ID+' node 1','ID','name'),'y','force',SW,'#',str(i.ID)+' node 1 load'],
            [common.df_value(i.nodes,i.ID+' node 2','ID','name'),'y','force',SW,'#',str(i.ID)+' node 2 load'],
        ],columns=['node','direction','type','value','#','ID'])
        load_joint = pd.concat([load_joint,load_temp])
    load = pd.concat([load,load_joint])
    
load = load.reset_index(drop=True)    
for i in range(len(load)):
    all_lines = add(load.loc[[i]].to_string(header=False, index=False))
load

,node,direction,type,value,#,ID
0,1,y,force,-191555.9375,#,beam node support 1
1,2,y,force,-1961.8750,#,beam node 2
2,3,y,force,-1961.8750,#,beam node 3
3,4,y,force,-1961.8750,#,beam node 4
4,5,y,force,-1961.8750,#,beam node 5
...,...,...,...,...,...,...
140,477,y,force,-1961.8750,#,joint 2 node 1 load
141,478,y,force,-1961.8750,#,joint 2 node 2 load
142,625,y,force,-1961.8750,#,joint 3 node 89 load
143,553,y,force,-1961.8750,#,joint 3 node 1 load


## analysis parameters

## conditions

In [29]:
all_lines = add('#')
cond = [['conditions'],
['lf.cnd.name limits'],
['c_lf', 0, 10],
['disp.cnd.name direction limits'],
['c_d_x','x',-2000,2000],
['c_d_y','y',-2000,2000],
['#']]
analysis = cond

### phases

In [30]:
phases = [['phases'],
['load.control'],
['increment path steps'],
[0.2, 'k', 50],
['displacement.control'],
['nod.name direction increment path steps'],
[82, 'y', 2000, 'c', 300],
['direction increment path steps #Arc length method if node omitted'],
['x+y', 1000, 'c', 300],
['automatic.control'],
['type path cnd.name'],
['nod.control translation c c_lf c_d_x c_d_y'],
['#']]
analysis.extend(phases)

### iterative strategy

In [31]:
iterative_str = [['iterative.strategy'],
['number.of.iterations = ',20],
['initial.reformations = ',20, '#if equal to number.of.iterations it is Full N-R'],
['step.reduction = ',10, '# step reduction'],
['divergence.iteration = ',10, '# no of iteration after which divergence is checked'],
['maximum.convergence = ',1.00E+20, '# iterations terminated if this convergence is exceeded'],
['#']]
analysis.extend(iterative_str)

### convergence criteria

In [32]:
convergance = [['convergence.criteria'],
['tolerance = ',1.00E-03, '# tolerance against which the convergence norm is compared'],
['work.ref = ',1.60E+07,'#work reference'],
['# force.ref = 1.00E+07 # used with the norm of out-of-balance forces to determine convergence'],
['# moment.ref = 1.00E+05 # used with the norm of out-of-balance moments to determine convergence'],
['#']]
analysis.extend(convergance)

### output

In [33]:
freq = 2 # results every `freq` output
output = [['output'],
['frequency '+str(freq)+' stress # output results for all equilibrium step, including step reductions'],
['#'],
['end']]
analysis.extend(output)

### print all the analysis parameters

In [34]:
for i in analysis:
    all_lines = add(common.str_joint(i))

## save file

Convert `all_lines` variable to a _*.dat_ file.

In [35]:
if save_file:
    f= open(project_name+".dat","w+")
    for i in all_lines:
         f.write(i[0]+"\n")
    f.close()
    print("File {0}.dat has been succesfully created.".format(project_name))

File 2plt3col.dat has been succesfully created.


In [36]:
if save_pickle:
    import pickle
    model = [material_lst,section_lst,group_lst,nodes,non_st_nodes,restraints,elm_cbp,elm_jel,lnk_lst,load]
    with open(project_name+'.pkl', "wb") as f:
        pickle.dump(model, f)
             
    f.close()    
    print('Saved pickle as '+project_name+'.pkl')

Saved pickle as 2plt3col.pkl
